In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, time, re
from datetime import datetime

In [ ]:
class General :
    
    @staticmethod
    def get_file_name( path ) :
        head, tail = ntpath.split( path )
        return tail or ntpath.basename( head )
    
    @staticmethod
    def print_info(iterable):
        print('Length is : ', len(iterable))
        print('Type is : ', type(iterable))
        print('Type of iterable[0] is : ', type(iterable[0]))
        print('That iterable is : ', iterable[0])
        
    @staticmethod
    def print_len_type(iterable):
        print('Length is : ', len(iterable))
        print('Type is : ', type(iterable))  
                                                      
    def read_chat_file(filePath):
        with open(filePath, encoding = 'utf8') as fin:
            r = fin.readlines()
        return r


In [ ]:
filePath = 'Friend.txt'

In [ ]:
class read:
    def make_DataFrames(filePath):

        words = []
        eachWordMetaData = []
        eachMessageMetaData = []
        people = set()
        lines = General.read_chat_file(filePath)

        # removing the unwanted lines
        for l in lines[:]:
            if '/' not in l:
                lines.remove(l)

        # splitting the lines into lists that have words
        for l in lines:
            words.append(l.split())

        # cleaning the date and names, making everything lowercase, removing hyphens
        for i in words:
            i[0] = i[0].strip(',')
            i[3] = i[3].strip(':')
            for j in range(len(i)):
                i[j] = i[j].lower()
            i.remove(i[2])

        # removing useless whatsapp info
        for i in words:
            if i[2] == 'messages':
                words.remove(i)

        # figuring out who the people are by taking the 2 names involved in the first 100 texts
        for i in words[:100]:
            people.add(i[2])
        person1, person2 = people

        # Changing their datatypes, now that names are known, so as to make counting easier
        # as ints are faster to iterate over than strings. Converting the date to its datetime format.
        for i in words:
            if i[2] == person1:
                i[2] = 1
            else: i[2] = 2
            i[0] = datetime.strptime(i[0] + ' ' + i[1], '%m/%d/%y %H:%M')
            i.remove(i[1])

        # Making a list with just the messages and its metadata, just the words and their metadata.
        for i in words:
            message = ''
            for j in range(2, len(i)):
                eachWordMetaData.append([i[1], i[0], i[j]])
                message += i[j] + ' '
            eachMessageMetaData.append([i[1], i[0], message.strip(' ')])

        # Making DataFrames for each person for words and messages using the lists made above.
        wordDf = pd.DataFrame(eachWordMetaData, columns = ['Speaker', 'Date', 'Word'])
        wordDf_P1 = wordDf[wordDf['Speaker'] == 1]
        wordDf_P2 = wordDf[wordDf['Speaker'] == 2]
        messageDf = pd.DataFrame(eachMessageMetaData, columns = ['Speaker', 'Date', 'Message'])
        messageDf_P1 = messageDf[messageDf['Speaker'] == 1]
        messageDf_P2 = messageDf[messageDf['Speaker'] == 2]


# Calculating the Statistics

In [ ]:
# making a statistics list to give back to the UI, that has all the basic info. 
# rest of the stuff is just basic data collection from dataframes. Nothing imp to explain.

statisticsList = []

# Messages Data
totalMessages = len(messageDf)
totalMessages_P1 = len(messageDf_P1)
totalMessages_P2 = len(messageDf_P2)
messagesPerc_P1 = round((totalMessages_P1/totalMessages) * 100, 2)
messagesPerc_P2 = round((totalMessages_P2/totalMessages) * 100, 2)

# Words Data
totalWords = len(wordDf)
totalWords_P1 = len(wordDf_P1)
totalWords_P2 = len(wordDf_P2)
wordsPerc_P1 = round((totalWords_P1/totalWords) * 100, 2)
wordsPerc_P2 = round((totalWords_P2/totalWords) * 100, 2)

# Unique Words
uniqueWords_P1 = wordDf_P1['Word'].unique()
uniqueWords_P2 = wordDf_P2['Word'].unique()
uniqueWords = wordDf['Word'].unique()

# now some Dates and times
totalMinutes = len(messageDf['Date'].unique())
uniqueDates = list(messageDf['Date'].apply(lambda x : x.replace(hour = 0, minute = 0, second = 0)).unique())
totalDays = len(uniqueDates)

# Finding the total time spent talking, and setting the right unit for it
totalTime = 0
timeUnit = 'M'
if totalMinutes > 60:
    totalTime = totalMinutes / 60
    timeUnit = 'H'
elif totalMinutes > 1440 :
    totalTime = totalMinutes/1440
    timeUnit = 'D'
else : totalTime = totalMinutes
totalTime = round(totalTime, 2)

# Finding Unique Words used by each, excluding emojis, and then finding most used words.
wordUsage_P1 = wordDf_P2['Word'].value_counts()
wordUsage_P2 = wordDf_P1['Word'].value_counts()
for i in wordUsage_P1.index:
    if i == '😂':
        wordUsage_P2.pop(i)
for i in wordUsage_P2.index:
    if i == '😂':
        wordUsage_P2.pop(i)
mostUsedWord_P1 = wordUsage_P1.index[0]
mostUsedWord_P2 = wordUsage_P2.index[0]

# finding the days without talking
dateDifferences = []

for i in range(1, len(uniqueDates)):
    dateDifferences.append(uniqueDates[i] - uniqueDates[i-1])
dateDifferences = np.array(dateDifferences).astype('timedelta64[h]')
for i in range(len(dateDifferences)):
    dateDifferences[i] = dateDifferences[i]/24
dateDifferences = dateDifferences.astype('int64')
maxDaysNoTalk = dateDifferences.max()

# Finding the number of days talked continuously.
maxDaysTalk = 0
count = 0
for i in range(1, len(dateDifferences)):
    if dateDifferences[i] == dateDifferences[i-1]:
        count += 1
        if count >= maxDaysTalk:
            maxDaysTalk = count
    else : count = 1

# Finding the maximum time spent talking in one day
# since you cant just take hours, as people talk in different hours wont mean they talk that much time, 
# we would take minutes, and then divide them to get hours
uniqueMinutes = messageDf['Date'].unique()
count = 1
maxMinutes = 0
for i in range(1, len(uniqueMinutes)):
    if uniqueMinutes[i].astype('datetime64[D]') == uniqueMinutes[i-1].astype('datetime64[D]'):
        count += 1
        if count >= maxMinutes:
            maxMinutes = count
    else: count = 1

# Setting the right time unit for it
maxTime = 0
timeUnit = 'M'
if maxMinutes > 60:
    maxTime = maxMinutes / 60
    timeUnit = 'H'
else : maxTime = maxMinutes
maxTime = round(maxTime, 2)

In [ ]:
# Appending the general stats to the list to give back to the UI

statisticsList.append('Total Number of Messages between both is : ' + str(totalMessages) + ', and words is : ' + str(totalWords))
if timeUnit == 'M':
    statisticsList.append('Total Number of Minutes spent talking is : ' + str(totalTime))
elif timeUnit == 'H':
    statisticsList.append('Total Number of Hours spent talking is : ' + str(totalTime))
elif timeUnit == 'D':
    statisticsList.append('Total Number of Days spent talking is : ' + str(totalTime))


statisticsList.append('Average time spent Talking Daily is : ' + str(round(totalMinutes/totalDays)) + ' Minutes')
statisticsList.append('Total Messages sent by ' + person1 + ' is : ' + str(totalMessages_P1) + ' which is ' + str(messagesPerc_P1) + ' %')
statisticsList.append('Total Messages sent by ' + person2 + ' is : ' + str(totalMessages_P2) + ' which is ' + str(messagesPerc_P2) + ' %')
statisticsList.append('Total Words sent by ' + person1 + ' is : ' + str(totalWords_P1) + ' which is ' + str(wordsPerc_P1) + ' %')
statisticsList.append('Total Words sent by ' + person2 + ' is : ' + str(totalWords_P2) + ' which is ' + str(wordsPerc_P2) + ' %')
statisticsList.append('Average Length of Each message of ' + person1 + ' is ' + str(round(totalWords_P1/totalMessages_P1)) + ' Words')
statisticsList.append('Average Length of Each message of ' + person2 + ' is ' + str(round(totalWords_P2/totalMessages_P2)) + ' Words')

statisticsList.append('Total Unique words used are ' + str(len(uniqueWords)) + ', ' + str(len(uniqueWords_P1)) + ' by ' + person1 + ' and ' + str(len(uniqueWords_P2)) + ' by ' + person2)
statisticsList.append('The Most used word by ' + person1 + ' is ' + '\"' + mostUsedWord_P1.upper() + '\"' + ', by ' + person2 + ' is ' + '\"' + mostUsedWord_P2.upper() + '\"')

statisticsList.append('The Most Days without talking were : ' + str(maxDaysNoTalk))
statisticsList.append('The Most Days talking consecutively were : ' + str(maxDaysTalk))
if timeUnit == 'M':
    statisticsList.append('Maximum number of Minutes spent talking in one day is : ' + str(maxTime))
elif timeUnit == 'H':
    statisticsList.append('Maximum number of Hours spent talking in one day is : ' + str(maxTime))

In [ ]:
statisticsList

# Messages Daily

In [ ]:
uniqueDates_P1 = messageDf_P1['Date'].apply(lambda x : x.replace(hour = 0, minute = 0, second = 0))
uniqueDates_P2 = messageDf_P2['Date'].apply(lambda x : x.replace(hour = 0, minute = 0, second = 0))
uniqueDates_Both = messageDf['Date'].apply(lambda x : x.replace(hour = 0, minute = 0, second = 0))

# Making DataFrames for Messages Sent Each Day
messagesEachDay_P1 = uniqueDates_P1.value_counts()
data = {
    'Dates' : list(messagesEachDay_P1.index),
    'Number of messages' : list(messagesEachDay_P1)
}
messagesEachDayDf_P1 = pd.DataFrame(data).sort_values(by = ['Dates'])

messagesEachDay_P2 = uniqueDates_P2.value_counts()
data = {
    'Dates' : list(messagesEachDay_P2.index),
    'Number of messages' : list(messagesEachDay_P2)
}
messagesEachDayDf_P2 = pd.DataFrame(data).sort_values(by = ['Dates'])

messagesEachDay_Both = uniqueDates_Both.value_counts()
data = {
    'Dates' : list(messagesEachDay_Both.index),
    'Number of messages' : list(messagesEachDay_Both)
}
messagesEachDayDf_Both = pd.DataFrame(data).sort_values(by = ['Dates'])


plt.style.use('fivethirtyeight')
plt.plot(messagesEachDayDf_P1['Dates'], messagesEachDayDf_P1['Number of messages'], label = person1, color = '#367ee2', linewidth = 1)
plt.plot(messagesEachDayDf_P2['Dates'], messagesEachDayDf_P2['Number of messages'], label = person2, color = '#3ba566', linewidth = 1)
plt.plot(messagesEachDayDf_Both['Dates'], messagesEachDayDf_Both['Number of messages'], label = 'Both', color = '#5f4b95', linewidth = 1)
plt.gcf().autofmt_xdate()
plt.title(f'{person1} with {person2} Messages Sent Daily')
plt.legend(loc = 'best')
plt.savefig(f'{person1} with {person2} Messages Sent Daily.jpg', dpi = 300)

# Words Daily

In [ ]:
uniqueDates_P1 = wordDf_P1['Date'].apply(lambda x : x.replace(hour = 0, minute = 0, second = 0))
uniqueDates_P2 = wordDf_P2['Date'].apply(lambda x : x.replace(hour = 0, minute = 0, second = 0))
uniqueDates_Both = wordDf['Date'].apply(lambda x : x.replace(hour = 0, minute = 0, second = 0))

# Making DataFrames for words Sent Each Day
wordsEachDay_P1 = uniqueDates_P1.value_counts()
data = {
    'Dates' : list(wordsEachDay_P1.index),
    'Number of words' : list(wordsEachDay_P1)
}
wordsEachDayDf_P1 = pd.DataFrame(data).sort_values(by = ['Dates'])

wordsEachDay_P2 = uniqueDates_P2.value_counts()
data = {
    'Dates' : list(wordsEachDay_P2.index),
    'Number of words' : list(wordsEachDay_P2)
}
wordsEachDayDf_P2 = pd.DataFrame(data).sort_values(by = ['Dates'])

wordsEachDay_Both = uniqueDates_Both.value_counts()
data = {
    'Dates' : list(wordsEachDay_Both.index),
    'Number of words' : list(wordsEachDay_Both)
}
wordsEachDayDf_Both = pd.DataFrame(data).sort_values(by = ['Dates'])


plt.style.use('fivethirtyeight')
plt.plot(wordsEachDayDf_P1['Dates'], wordsEachDayDf_P1['Number of words'], label = person1, color = '#367ee2', linewidth = 1)
plt.plot(wordsEachDayDf_P2['Dates'], wordsEachDayDf_P2['Number of words'], label = person2, color = '#3ba566', linewidth = 1)
plt.plot(wordsEachDayDf_Both['Dates'], wordsEachDayDf_Both['Number of words'], label = 'Both', color = '#5f4b95', linewidth = 1)
plt.gcf().autofmt_xdate()
plt.title(f'{person1} with {person2} words Sent Daily')
plt.legend(loc = 'best')
plt.savefig(f'{person1} with {person2} words Sent Daily.jpg', dpi = 300)

# Percentage of Messages

In [ ]:
plt.pie([messagesPerc_P1,  messagesPerc_P2], startangle = 90, labels = [person1, person2], colors = ['#367ee2', '#3ba566'], shadow = True, autopct = '%1.2f%%')
plt.legend(loc = 'best')
plt.title(f'{person1} and {person2} Percentage of Messages Sent')
plt.savefig(f'{person1} and {person2} Percentage of Messages Sent', dpi = 300)

# Percentage of Words

In [ ]:
plt.pie([wordsPerc_P1,  wordsPerc_P2], startangle = 90, labels = [person1, person2], colors = ['#367ee2', '#3ba566'], shadow = True, autopct = '%1.2f%%')
plt.legend(loc = 'best')
plt.title(f'{person1} and {person2} Percentage of Words Sent')
plt.savefig(f'{person1} and {person2} Percentage of Words Sent', dpi = 300)

# Messages Monthly

In [ ]:
uniqueDates_P1 = messageDf_P1['Date'].apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0))
uniqueDates_P2 = messageDf_P2['Date'].apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0))
uniqueDates_Both = messageDf['Date'].apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0))

# Making DataFrames for Messages Sent Each Day
messagesEachDay_P1 = uniqueDates_P1.value_counts()
data = {
    'Dates' : list(messagesEachDay_P1.index),
    'Number of messages' : list(messagesEachDay_P1)
}
messagesEachDayDf_P1 = pd.DataFrame(data).sort_values(by = ['Dates'])

messagesEachDay_P2 = uniqueDates_P2.value_counts()
data = {
    'Dates' : list(messagesEachDay_P2.index),
    'Number of messages' : list(messagesEachDay_P2)
}
messagesEachDayDf_P2 = pd.DataFrame(data).sort_values(by = ['Dates'])

messagesEachDay_Both = uniqueDates_Both.value_counts()
data = {
    'Dates' : list(messagesEachDay_Both.index),
    'Number of messages' : list(messagesEachDay_Both)
}
messagesEachDayDf_Both = pd.DataFrame(data).sort_values(by = ['Dates'])


plt.style.use('fivethirtyeight')
plt.plot(messagesEachDayDf_P1['Dates'], messagesEachDayDf_P1['Number of messages'], label = person1, color = '#367ee2', linewidth = 1)
plt.plot(messagesEachDayDf_P2['Dates'], messagesEachDayDf_P2['Number of messages'], label = person2, color = '#3ba566', linewidth = 1)
plt.plot(messagesEachDayDf_Both['Dates'], messagesEachDayDf_Both['Number of messages'], label = 'Both', color = '#5f4b95', linewidth = 1)
a = [i.to_pydatetime().strftime('%B') for i in messagesEachDayDf_P1['Dates']]
plt.xticks(labels = a, ticks = [i.to_pydatetime() for i in messagesEachDayDf_P1['Dates']])
plt.gcf().autofmt_xdate()
plt.title(f'{person1} with {person2} Messages Sent Monthly')
plt.legend(loc = 'best')
plt.savefig(f'{person1} with {person2} Messages Sent Monthly.jpg', dpi = 300)

# Words Sent Monthly

In [ ]:
wordDatesMonthly_P1 = wordDf_P1['Date'].apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0))
wordDatesMonthly_P2 = wordDf_P2['Date'].apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0))
wordDatesMonthly_Both = wordDf['Date'].apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0))

# Making DataFrames for words Sent Each Day
wordsEachDay_P1 = wordDatesMonthly_P1.value_counts()
data = {
    'Dates' : list(wordsEachDay_P1.index),
    'Number of words' : list(wordsEachDay_P1)
}
wordsEachDayDf_P1 = pd.DataFrame(data).sort_values(by = ['Dates'])

wordsEachDay_P2 = wordDatesMonthly_P2.value_counts()
data = {
    'Dates' : list(wordsEachDay_P2.index),
    'Number of words' : list(wordsEachDay_P2)
}
wordsEachDayDf_P2 = pd.DataFrame(data).sort_values(by = ['Dates'])

wordsEachDay_Both = wordDatesMonthly_Both.value_counts()
data = {
    'Dates' : list(wordsEachDay_Both.index),
    'Number of words' : list(wordsEachDay_Both)
}
wordsEachDayDf_Both = pd.DataFrame(data).sort_values(by = ['Dates'])


plt.style.use('fivethirtyeight')
plt.plot(wordsEachDayDf_P1['Dates'], wordsEachDayDf_P1['Number of words'], label = person1, color = '#367ee2', linewidth = 1.5)
plt.plot(wordsEachDayDf_P2['Dates'], wordsEachDayDf_P2['Number of words'], label = person2, color = '#3ba566', linewidth = 1.5)
plt.plot(wordsEachDayDf_Both['Dates'], wordsEachDayDf_Both['Number of words'], label = 'Both', color = '#5f4b95', linewidth = 1.5)
a = [i.to_pydatetime().strftime('%B') for i in wordsEachDayDf_P1['Dates']]
plt.xticks(labels = a, ticks = [i.to_pydatetime() for i in wordsEachDayDf_P1['Dates']])
plt.gcf().autofmt_xdate()
plt.title(f'{person1} with {person2} words Sent Monthly')
plt.legend(loc = 'best')
plt.savefig(f'{person1} with {person2} words Sent Monthly.jpg', dpi = 300)

# 20 Most used Words

In [ ]:
wordDatesMonthly_P1 = wordDf_P1['Date'].apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0))
wordDatesMonthly_P2 = wordDf_P2['Date'].apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0))

wordsEachDay_P1 = wordDatesMonthly_P1.value_counts()
data = {
    'Words' : list(wordUsage_P1.index),
    'Number' : list(wordUsage_P1)
}
wordUsageDf_P1 = pd.DataFrame(data).sort_values(by = ['Number'], ascending = False)

wordUsageDf_P2 = wordDatesMonthly_P2.value_counts()
data = {
    'Words' : list(wordUsage_P2.index),
    'Number' : list(wordUsage_P2)
}
wordUsageDf_P2 = pd.DataFrame(data).sort_values(by = ['Number'], ascending = False)

num_of_words = 20

p1x = np.arange(num_of_words)
p2x = np.arange(num_of_words) + np.array(0.3)
plt.figure(figsize=(20, 7))
plt.style.use('fivethirtyeight')
plt.bar(p1x, wordUsageDf_P1['Number'][:num_of_words], label = person1, color = '#9d63cf', width = 0.3, edgecolor = 'k')
plt.bar(p2x, wordUsageDf_P2['Number'][:num_of_words], label = person2, color = '#ecac74', width = 0.3, edgecolor = 'k')
l = []
for i in range(num_of_words):
    l.append(wordUsageDf_P1['Words'][:num_of_words][i])
    l.append(wordUsageDf_P2['Words'][:num_of_words][i])

t = np.vstack((p1x, p2x)).reshape((-1,),order='F')
plt.xticks(ticks = t, labels = l, rotation = 45)
plt.title(f'{person1} with {person2} Top 20 Words')
plt.legend(loc = 'best')
plt.savefig(f'{person1} with {person2} Top 20 Words.jpg', dpi = 300)

In [ ]:
pd.Series(uniqueDates)

In [ ]:
just_dates = pd.Series(messageDf['Date'].unique()).apply(lambda x : x.replace(hour = 0, minute = 0, second = 0))
HoursDaily = (just_dates.value_counts()/60).sort_index()
# a = pd.Series([i.to_pydatetime().strftime('%b') for i in HoursDaily.index]).unique()
plt.plot(HoursDaily.index, HoursDaily, color = '#9d63cf', linewidth = 1.5)
plt.title(f'{person1} with {person2} Time Spent Daily')
plt.xlabel('Dates')
plt.ylabel('Hours')
plt.gcf().autofmt_xdate()
plt.savefig(f'{person1} with {person2} Time Spent Daily.jpg', dpi = 300)

In [ ]:
just_dates = pd.Series(pd.Series(messageDf['Date'].unique()).apply(lambda x : x.replace(day = 1, hour = 0, minute = 0, second = 0)))
HoursMonthly = (just_dates.value_counts()/60).sort_index()
a = pd.Series([i.to_pydatetime().strftime('%B-%Y') for i in HoursMonthly.index])
plt.bar(a, HoursMonthly, color = '#71194b', linewidth = 1.5)
plt.title(f'{person1} with {person2} Time Spent Monthly')
plt.xlabel('Months')
plt.ylabel('Hours')
plt.gcf().autofmt_xdate()
plt.savefig(f'{person1} with {person2} Time Spent Monthly.jpg', dpi = 300)

# Usage of Particular Words Over time

In [ ]:
words = ['and', 'may', 'hi', 'the', 'a', 'it', 'sleep', 'night', 'morning', 'die', 'hell', 'that', 'yea', 'Friend']
colors = ['#296a30', '#1d8baf', '#c94a4a', '#9d63cf']
import math
for i in range(math.ceil(len(words)/4)):
    for _, word in enumerate(words[i*4:(i+1)*4], start = 0):
        try:
            particularWordDf = wordDf[wordDf['Word'] == word]
            particularWordUsage = particularWordDf['Date'].apply(lambda x : x.replace(hour = 0, minute = 0, second = 0)).value_counts().sort_index()
            plt.plot(particularWordUsage, color = colors[_], linewidth = 1, label = word)
            plt.title(f'{person1} and {person2} Special Word Usages - ' + str(i+1))
            plt.legend(loc = 'best')
            plt.xlabel('Dates')
            plt.ylabel('Number of Times Used')
            plt.gcf().autofmt_xdate()
        except IndexError as err:
            print(err)
            break
    plt.savefig(f'{person1} and {person2} Special Word Usages - ' + str(i+1) + '.jpg', dpi = 300)
    plt.clf()

# Length of Each Session Histogram

In [ ]:
uniqueMinutes = messageDf['Date'].unique()
bins = list(np.arange(0, 10, step = .5))
bins = [1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 90, 120, 150, 180, 210, 240, 300, 1000]

uniqueMinutes = pd.Series(uniqueMinutes).sort_values()
count = 1
sessions = []
for i in range(len(uniqueMinutes)-1):
    if uniqueMinutes[i].to_pydatetime().strftime('%y%m%d%h') == uniqueMinutes[i].to_pydatetime().strftime('%y%m%d%h'):
        if uniqueMinutes[i].minute >= uniqueMinutes[i+1].minute - 5:
            count += 1
        else :
            sessions.append(count)
            count = 1
h,e = np.histogram(sessions, bins=bins)
plt.bar(range(len(bins)-1),h, width=1, edgecolor='#296a30',  color = '#9bee81')
labels = []
labels.append('<1')
for i in range(1, 13 ):
    labels.append(str(bins[i]) + '-' + str(bins[i+1]))
for i in ['1 Hr', '1.5 Hr', '2 Hr', '2.5 Hr', '3 Hr', '3.5 Hr', '4 Hr', '5 Hr', '5+ Hrs']:
    labels.append(i)
plt.xlabel('Minutes')
plt.ylabel('Number of Sessions')
plt.title(f'{person1} with {person2} Length of Texting Sessions')
plt.xticks(ticks = range(len(bins)-1), labels = labels, rotation = 45)
plt.savefig(f'{person1} and {person2} Session Times.jpg', dpi = 300)


# Time Spent on Each WeekDay

In [ ]:
uniqueMinutes = pd.Series(messageDf['Date'].unique()).sort_values()
weekdays = [0 for i in range(7)]
for i in range(len(uniqueMinutes)):
    if uniqueMinutes[i].day_name() == 'Monday':
        weekdays[0] += 1
    elif uniqueMinutes[i].day_name() == 'Tuesday':
        weekdays[1] += 1
    elif uniqueMinutes[i].day_name() == 'Wednesday':
        weekdays[2] += 1
    elif uniqueMinutes[i].day_name() == 'Thursday':
        weekdays[3] += 1
    elif uniqueMinutes[i].day_name() == 'Friday':
        weekdays[4] += 1
    elif uniqueMinutes[i].day_name() == 'Saturday':
        weekdays[5] += 1
    elif uniqueMinutes[i].day_name() == 'Sunday':
        weekdays[6] += 1

weekdays = [i/60 for i in weekdays]
plt.bar(range(7), weekdays, color = '#ecac74')
plt.title(f'{person1} with {person2} Total Hours Spent over the Week')
plt.xlabel('WeekDays')
plt.ylabel('Hours')
plt.xticks(labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ticks = np.arange(7), rotation = 10)
plt.savefig(f'{person1} with {person2} Total Time spent Over the Week.jpg', dpi = 300)

# Most Active Hours

In [ ]:
messagesEachHour = messageDf['Date'].apply(lambda x : x.replace(day = 1, month = 1, year = 2020, minute = 0, second = 0)).value_counts().sort_index()
pd.Series(messagesEachHour.index)
hours = [i.hour for i in pd.Series(messagesEachHour.index)]

In [ ]:
plt.bar(hours, messagesEachHour, color = '#ecac74')
plt.title(f'{person1} with {person2} Total Messages Each Hour All Time')
plt.xlabel('Hours')
plt.ylabel('Number of Messages')
plt.savefig(f'{person1} with {person2} Total Messages Each Hour All Time.jpg', dpi = 300)

In [ ]:
date_regex = re.compile(r"((\d|1[0-2])/[0-2]\d|3[01])/([12]\d{1})")
match = date_regex.search('1/10/21, 16:32 - Friend: Very nice sentence')
print(match)

In [ ]:
"31/02/2020"